In [40]:
import os
import cv2
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import numpy as np

In [41]:
save_panoptic_label_dir_name = 'target1'
save_panoptic_data_dir_name = 'input1'
save_dir = 'validation_mapillary'
save_panoptic_label_dir_path = os.path.join(save_dir, save_panoptic_label_dir_name)
save_panoptic_data_dir_path = os.path.join(save_dir, save_panoptic_data_dir_name)

In [42]:
print(save_panoptic_label_dir_path, save_panoptic_data_dir_path)

validation_mapillary\target1 validation_mapillary\input1


In [43]:
os.makedirs(save_dir, exist_ok=True)
os.makedirs(save_panoptic_label_dir_path, exist_ok=True)
os.makedirs(save_panoptic_data_dir_path, exist_ok=True)

In [44]:
target_color = np.array([60, 20, 220])
original_folder = "validation_mapillary"
original_input_folder = f'{original_folder}\input'
original_target_folder = f'{original_folder}' + r'\t' + 'arget'
print(original_input_folder, original_target_folder)

validation_mapillary\input validation_mapillary\target


In [45]:
print(len(os.listdir(original_input_folder)))

2000


In [46]:
def process_file(filename):
    if not (filename.endswith('.jpg') or filename.endswith('.png')):
        return
    label_filename = os.path.splitext(filename)[0] + '.png'
    input_path = os.path.join(original_input_folder, filename)
    label_path = os.path.join(original_target_folder, label_filename)

    label_img = cv2.imread(label_path)
    if label_img is None:
        return
    mask = np.all(label_img == target_color, axis=2).astype(np.uint8) * 255
    if np.any(mask):
        new_input_path = os.path.join(save_panoptic_data_dir_path, filename)
        cv2.imwrite(new_input_path, cv2.imread(input_path))

        new_label_path = os.path.join(save_panoptic_label_dir_path, label_filename)
        cv2.imwrite(new_label_path, mask)

In [47]:
filenames = os.listdir(original_input_folder)

with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    list(tqdm(executor.map(process_file, filenames), total=len(filenames)))

100%|██████████| 2000/2000 [01:20<00:00, 24.97it/s]
